<a href="https://colab.research.google.com/github/520817/gp/blob/main/adhd_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(42)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.cuda.is_available()

False

In [ ]:
torch.cuda.get_device_name(0)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
!git clone https://github.com/kmounlp/NER.git

In [ ]:
import os
import glob

In [ ]:
file_list=[]

In [ ]:
for x in os.walk('NER/'):
  for y in glob.glob(os.path.join(x[0], '*_NER.txt')):
    file_list.append(y)

In [ ]:
file_list = sorted(file_list)

In [ ]:
from pathlib import Path

file_path = file_list[0]
file_path = Path(file_path)
raw_text = file_path.read_text().strip()

In [ ]:
!pip install korpora

from Korpora import Korpora
corpus = Korpora.load("naver_changwon_ner")

네이버 데이터셋 전처리

In [ ]:
def naver_read_file(file_list):

    token_docs = []
    tag_docs = []

    for doc in file_list:
        tokens = []
        tags = []
        list1=doc.words
        list2=doc.tags

        for text,tag in zip(list1,list2):
            for i, syllable in enumerate(text): # 음절 단위로 자르고
                tokens.append(syllable)
                if tag == '-':
                  tag = 'O' # 태그가 '-'인 경우 'O'로 변경
                tags.append(tag)

        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
naver_text,naver_tags =naver_read_file(corpus.train)

In [ ]:
print(len(naver_text))
print(len(naver_tags))

In [ ]:
print(naver_text[0], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(naver_tags[0])

해양 뭐시기 데이터셋 전처리

In [ ]:
import re
def read_file(file_list):
    token_docs = []
    tag_docs = []
    for file_path in file_list:
        #print("read file from ", file_path)
        file_path = Path(file_path)
        raw_text = file_path.read_text().strip()
        raw_docs = re.split(r'\n\t?\n', raw_text)

        for doc in raw_docs:
            tokens = []
            tags = []
            for line in doc.split('\n'):
                if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
                    continue
                token = line.split('\t')[0]
                tag = line.split('\t')[3]
                for i, syllable in enumerate(token):    # 음절 단위로 잘라서
                    tokens.append(syllable)
                    tags.append(tag)

            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
texts, tags = read_file(file_list[:])

In [ ]:
print(len(texts))
print(len(tags))

In [ ]:
print(texts[0], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(tags[0])

분리된 토큰을 하나의 개체명으로 묶어 주는 태깅 시스템을 위해 있는게 BIO, BIESO임.

여러 개의 토큰으로 이루어진 개체명의 경우

**개체명이 시작할 때** 'B,begin'

**토큰이 개체명 중간에 있을 때** 'I,inside'

**개체명의 마지막에 위치할 때** 'E,end'

**하나의 토큰이 곧 하나의 개체명일 때** 'S,singleton'

**토큰이 개체명이 아닐 경우** 'O,outside'

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 전

In [ ]:
for sublist in tags:
    for i, label in enumerate(sublist):
        if label == 'I-NOH':
            sublist[i] = 'NOH_I'
        elif label == 'I-PNT':
            sublist[i] = 'PNT_I'
        elif label == 'I-ORG':
            sublist[i] = 'ORG_I'
        elif label == 'I-DUR':
            sublist[i] = 'DUR_I'
        elif label == 'I-TIM':
            sublist[i] = 'TIM_I'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'I-MNY':
            sublist[i] = 'MNY_I'
        elif label == 'B-MNY':
            sublist[i] = 'MNY_B'
        elif label == 'B-DAY':
            sublist[i] = 'DAY_B'
        elif label == 'B-TIM':
            sublist[i] = 'TIM_B'
        elif label == 'I-LOC':
            sublist[i] = 'LOC_I'
        elif label == 'B-PER':
            sublist[i] = 'PER_B'
        elif label == 'B-ORG':
            sublist[i] = 'ORG_B'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'B-PNT':
            sublist[i] = 'PNT_B'
        elif label == 'B-NOH':
            sublist[i] = 'NOH_B'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'I-POH':
            sublist[i] = 'POH_I'
        elif label == 'I-DAT':
            sublist[i] = 'DAT_I'
        elif label == 'B-POH':
            sublist[i] = 'POH_B'
        elif label == 'B-DUR':
            sublist[i] = 'DUR_B'
        elif label == 'B-DAT':
            sublist[i] = 'DAT_B'
        elif label == '0':
            sublist[i] = '0'

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 후

In [ ]:
texts.extend(naver_text)
tags.extend(naver_tags)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
for i, tag in enumerate(unique_tags):
    print(tag)  # 학습을 위한 label list를 확인합니다.

In [ ]:
for tag in list(tag2id.keys()) :
    globals()[tag] = 0

for tag in tags :
    for ner in tag :
        globals()[ner] += 1

for tag in list(tag2id.keys()) :
    print('{:>6} : {:>7,}'. format(tag, globals()[tag]))

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_tags, test_tags = train_test_split(texts, tags, test_size=.2, random_state=42)

In [ ]:
print('Train 문장 : {:>6,}' .format(len(train_texts)))
print('Train 태그 : {:>6,}' .format(len(train_tags)))
print('Test  문장 : {:>6,}' .format(len(test_texts)))
print('Test  태그 : {:>6,}' .format(len(test_tags)))

In [ ]:
train_texts[5]

BERT 토크나이저

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

In [ ]:
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [ ]:
print(ner_tokenizer(train_texts[0], 5))

In [ ]:
tokenized_train_sentences = []
tokenized_test_sentences = []

for text in train_texts:    # 전체 데이터를 tokenizing 합니다.
    tokenized_train_sentences.append(ner_tokenizer(text, 128))
for text in test_texts:
    tokenized_test_sentences.append(ner_tokenizer(text, 128))

In [ ]:
def encode_tags(tags, max_seq_length):
    # label 역시 입력 token과 개수를 맞춰줍니다
    tags = tags[:max_seq_length-2]
    labels = [tag2id[tag] for tag in tags]
    labels = [tag2id['O']] + labels

    padding_length = max_seq_length - len(labels)
    labels = labels + ([pad_token_label_id] * padding_length)

    return labels

In [ ]:
tag2id

In [ ]:
encode_tags(train_tags[0], 5)

In [ ]:
train_labels = []
test_labels = []

for tag in train_tags:
    train_labels.append(encode_tags(tag, 128))

for tag in test_tags:
    test_labels.append(encode_tags(tag, 128))

In [ ]:
len(train_labels), len(test_labels)

Token 데이터셋

In [ ]:
import torch

class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TokenDataset(tokenized_train_sentences, train_labels)
test_dataset = TokenDataset(tokenized_test_sentences, test_labels)

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments, AutoModelForTokenClassification,EarlyStoppingCallback
import sys
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000, # 1000번쨰 steps마다 log를 보여줌
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=5,
    save_strategy='steps', # steps로 해야 earlystop이 가능
    evaluation_strategy='steps',
    save_steps=1000, # 1000번쨰 step마다 저장
    eval_steps=1000, # 1000번째 step마다 평가
    seed=15,
    load_best_model_at_end=True # 가장 좋은 성능의 모델로...
)